# Stockage de données non institutionnel

Estimation du bilan carbone du stockage sur des drive/dropbox/etc par le personnel de l'UMR (hors stockage institutionnel type Ob7).

## Incertitudes

Beaucoup d'incertitude vient des points suivants:

- on a mal fait notre question, dans la réponse au questionnaire on a l'info de la quantité de stockage mais pas différenciée par type. En particulier, on ne sait pas si c'est du stockage sur drive/dropbox ou sur nas

- on a du mal à trouver un équivalent quantité de données stockées - consommation électrique (avec une source fiable). Pour le moment on n'a que ça: https://medium.com/stanford-magazine/carbon-and-the-cloud-d6f481b79dfe

- pour passer de la consommation électrique à un équivalent carbone, idéalement il faut savoir dans quel pays la donnée est stockée exactement (le ratio américain kgeqCO2/kWh est 6 fois supérieur au français par exemple)

## Localication physique du stockage 

Dropbox stock au Etats Unis : https://help.dropbox.com/accounts-billing/security/physical-location-data-storage

Google drive semble être identique par défaut : https://support.google.com/a/answer/9223653?visit_id=637794849382956724-3553915730&rd=1

OneDrive (Microsoft) semble plutôt localisé en Europe : https://docs.microsoft.com/fr-fr/microsoft-365/enterprise/eu-data-storage-locations?view=o365-worldwide

Donc pour le moment, comme on manque d'informations plus précises, on considère que les données sont stockées aux Etas Unis

In [1]:
import pandas as pd
import numpy as np
import re
#import math
#from datetime import timedelta

stock_df = pd.read_csv("data-donnees-internet/results-survey685737-stockage-2023.csv", sep=',', index_col=0)
stock_df

,Date de soumission,Dernière page,Langue de départ,Tête de série,Date de lancement,Date de la dernière action,Quel est votre nom?,Quel est votre statut?,Quel est votre statut? [Autre],"Utilisez-vous un ou plusieurs espace(s) de stockage en ligne NON MUTUALISÉ pour votre activité professionelle (Leviia, Dropbox, Google drive, autre NAS à distance, NAS de bureau etc...) ?",...,Rajouter une sortie.8,Informations aéronef [Nom][].4,Informations aéronef [Origine][].4,Type.4,Consommation [Puissance (en CV)].4,Consommation [Heures de vol totales].8,Consommation [Shaft power].4,Consommation [Heures de vol totales].9,Rajouter une sortie.9,"N'hésitez pas à nous faire part de vos remarques, suggestions, commentaires, avis ! Que ce soit au sujet de l'enquête en elle-même, le calcul du bilan C prévu de l'UMR, MARBEC Green ..."
ID de la réponse,,,,,,,,,,,,,,,,,,,,,
1,NaN,1,fr,1300608535,2024-02-09 15:07:22,2024-02-09 15:07:22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,fr,64237810,2024-02-14 21:39:39,2024-02-14 21:39:39,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1,fr,736959744,2024-02-16 09:20:45,2024-02-16 09:20:45,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-02-16 14:20:43,1,fr,1769548857,2024-02-16 14:19:41,2024-02-16 14:20:43,Antoine Fraimout,Docs. / Post-Docs.,NaN,Oui,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-02-16 14:21:13,1,fr,1085960557,2024-02-16 14:19:44,2024-02-16 14:21:13,Philippe CECCHI,Chercheur/Chercheuse,NaN,Non,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,NaN,1,fr,694988697,2024-03-14 06:15:34,2024-03-14 06:15:34,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,2024-03-14 07:21:26,1,fr,919544114,2024-03-14 07:19:04,2024-03-14 07:21:26,Hervé Demarcq,Chercheur/Chercheuse,NaN,Non,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,1,fr,967392693,2024-03-15 14:51:45,2024-03-15 14:51:45,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Extract the column of interest, i.e. the columns with the storage type and the storage capacity. We also append the Datarmor and Marbec-Data storage

In [2]:
colonne_type = 10
colonne_size = 11
df_small = stock_df.iloc[:, [colonne_type, colonne_size]]
df_small.columns = ['type', 'size']

# adding Datarmor 
n = len(df_small.index) + 2
temp = pd.DataFrame(data={'type': ['Datarmor'], 'size': [f'{int(80937.95 + 	439177.08)} Go']}, index=[n])
df_small = pd.concat([df_small, temp])

# adding Meso@LR 
n = len(df_small.index) + 2
temp = pd.DataFrame(data={'type': ['Marbec-Data'], 'size': [f'{int(51.4 * 1e3 + 17)} Go']}, index=[n])
df_small = pd.concat([df_small, temp])
df_small

,type,size
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
5,Google Drive,2 To
6,NaN,NaN
...,...,...
242,NaN,NaN
243,NaN,NaN
244,NaN,NaN
245,Datarmor,520115 Go


Maintenant on nettoie le fichier pour ne garder que ce qui est exploitable (valeurs numeriques et unites). Et on convertit tout en Giga

In [3]:
regex = re.compile('<? *([0-9]+) *([gGtTmM] ?[bo]?) *$')
index = []
size = []
for i in range(len(df_small.index)):
    l = df_small.iloc[i, 1]
    if regex.match(str(l)):

        groups = regex.match(l).groups()
        tempsize = float(groups[0])

        # converts size in Go
        if groups[1].startswith('m') | groups[1].startswith('M') :
            tempsize *= 1e-3
        if groups[1].startswith('T'):
            tempsize *= 1e3
            
        index.append(i)
        size.append(tempsize)
        

In [4]:
df_out = df_small.iloc[index].copy()
df_out['giga'] = size
df_out

,type,size,giga
5,Google Drive,2 To,2000.0
18,Google Drive,15Go,15.0
21,DropBox,2To,2000.0
26,Google drive,10Go,10.0
33,Time Capsule,1 To,1000.0
36,Disque dur externe,2 To,2000.0
40,Nextcloud,150Go,150.0
46,NAS à distance,1 To,1000.0
54,Google Drive,15 Go,15.0
65,Google drive,2 Go,2.0


Maintenant, en fonction du type de stockage, on attribut des valeurs d'emission par Go/an:

https://blog.lecko.fr/comment-mesurer-les-emissions-de-ges-liees-aux-usages-numeriques/

- Facteur US: 2 kgCO2e / Go / an.
- Facteur FR: 400 gCO2e / Go /an


In [5]:
index = []
ratio = []
conversion_us = 2  # kgCo2 /an/Go
conversion_fr = 0.4 # kGCo2/an/Go

ratio_fr = 0.112 # LOCEAN
ratio_us = 0.493
ratio_monde = 0.519

for type in df_out['type'].values:
    type = type.lower()
    if ('google' in type) or ('dropbox' in type):
        ratio.append(conversion_us)
    else:
        ratio.append(conversion_fr)
df_out.loc[:, 'conversion'] = ratio  
df_out

,type,size,giga,conversion
5,Google Drive,2 To,2000.0,2.0
18,Google Drive,15Go,15.0,2.0
21,DropBox,2To,2000.0,2.0
26,Google drive,10Go,10.0,2.0
33,Time Capsule,1 To,1000.0,0.4
36,Disque dur externe,2 To,2000.0,0.4
40,Nextcloud,150Go,150.0,0.4
46,NAS à distance,1 To,1000.0,0.4
54,Google Drive,15 Go,15.0,2.0
65,Google drive,2 Go,2.0,2.0


Maintenant on convertit les Giga en kilogrammes de CO2

In [6]:
output = df_out.loc[:, 'giga'] * df_out.loc[:, 'conversion']
output

5        4000.0
18         30.0
21       4000.0
26         20.0
33        400.0
36        800.0
40         60.0
46        400.0
54         30.0
65          4.0
67        736.0
69        260.0
76          2.0
86          2.0
87         30.0
95          6.0
98        160.0
102        30.0
106      8000.0
108       400.0
112       200.0
114      4000.0
119        20.0
125      1600.0
126      2000.0
136      1800.0
137         2.0
141       400.0
142        48.0
145        18.0
146      1200.0
152      4000.0
154       240.0
155         4.0
160         8.0
169      2000.0
185      4000.0
190        40.0
197      4000.0
198        20.0
199        30.0
201        30.0
202      4000.0
221        30.0
224        40.0
230         2.0
232        10.0
233         0.4
236       400.0
245    208046.0
246     20566.8
dtype: float64

Et on somme le tout pour avoir les kilos totales de CO2

In [7]:
totco2 = output.sum()
totco2 = totco2 * 1e-3
output = f'Emission totale de CO2 liee au stockage de CO2: {totco2} tons'
output

'Emission totale de CO2 liee au stockage de CO2: 278.1252 tons'

In [8]:
# stock_df = stock_df.replace({"oui":1,"non":0})
# prct = 100 * stock_df['Stockage'].sum() / stock_df.shape[0]
# print('Pourcentage des personnes avec un Drive/Dropbox/etc.:', prct)

In [9]:
# boolean1 = stock_df.loc[:,"Type"] != "NAS"
# boolean2 = stock_df.loc[:,"Stockage"] == 1`
# stock_df_1 = stock_df[boolean1 & boolean2]

# #stock_df_2 = stock_df[(boolean1 == False) | (boolean2 == False)]
# #stock_df_2.shape[0]

In [10]:
# moy_par_personne = (stock_df_1['Quantite_Tb'].sum(skipna = True))/(stock_df.shape[0])
# print('Quantite moyenne de donnees stockees (hors NAS):', moy_par_personne, "Tb")

In [11]:
# print('Quantite totale de donnees stockees (hors NAS, pour les personnes qui ont repondu):', stock_df_1['Quantite_Tb'].sum(skipna = True), "Tb")

In [12]:
# effectif = 265
# total_umr = moy_par_personne * effectif
# print('Quantite totale de donnees par l ensemble du personnel UMR:', total_umr, "Tb")

In [13]:
# eq_Gb_kWh = (7 + 3.1)/2 #kWh necessaire au stockage d'un Gb pendant un an 
# eq_Tb_kWh = eq_Gb_kWh * 10**3

# conso_stockage_umr = total_umr * eq_Tb_kWh
# print("Consommation totale de l'UMR liee au stockage de donnees:", conso_stockage_umr, "kWh")

## Passage kWh - carbone

Ratios (source = https://theshiftproject.org/en/carbonalyser-browser-extension/):

- European Union : 0,276 kgCO2e/kWh

- France : 0,035 kgCO2e/kWh

- United States : 0,493 kgCO2e/kWh

- China : 0,681 kgCO2e/kWh

- Other (average world carbon intensity factor) : 0,519 kgCO2e/kWh

In [14]:
# ratio_fr = 0.112 # LOCEAN
# ratio_us = 0.493
# ratio_monde = 0.519

# C_tot_fr = conso_stockage_umr * ratio_fr / 10**3
# C_tot_us = conso_stockage_umr * ratio_us / 10**3
# C_tot_monde = conso_stockage_umr * ratio_monde / 10**3

# print("eqCO2 total (ratio francais):",C_tot_fr,"t eqCO2")

In [15]:
# print("eqCO2 total (ratio US):",C_tot_us,"t eqCO2")

In [16]:
# print("eqCO2 total (ratio monde):",C_tot_monde,"t eqCO2")